<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/PyTorch%20Persian%20Mnist%20TL/Mnist_PersianTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
import wandb

In [2]:
wandb.init(project="PersianMnistTL", entity="ma_heravi")

wandb: Currently logged in as: ma_heravi (use `wandb login --relogin` to force relogin)


In [3]:
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

In [4]:
# hyperparameters
latent_size = 10
disc_inp_sz = 28*28
img_size = 28
epochs = 10
batch_size = 32
lr = 0.001
wandb.config = {
  "learning_rate": lr,
  "epochs": epochs,
  "batch_size": batch_size
}

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torchvision.models.resnet50(pretrained=True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features,10)

ct = 0
for child in model.children():
  ct +=1
  if ct < 7:
     for param in child.parameters():
         param.requires_grad = False

model = model.to(device)

In [8]:
def calc_acc(preds, labels):
    _, preds_max = torch.max(preds, 1)
    acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
    return acc

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Data Preparing

transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize((28, 28)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


dataset = torchvision.datasets.ImageFolder("/content/drive/MyDrive/MNIST_persian", transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [11]:
 # compile
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [12]:
# train
wandb.watch(model)

for epoch in range(1, epochs+1):
    train_loss = 0.0
    train_acc = 0.0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # 1- forwarding
        preds = model(images)
        # 2- backwarding 
        loss = loss_function(preds, labels)
        loss.backward()
        # 3- Update
        optimizer.step()

        train_loss += loss
        train_acc += calc_acc(preds, labels)
    
    total_loss = train_loss / len(train_data_loader)
    total_acc = train_acc / len(train_data_loader)

    print(f"Epoch: {epoch}, Loss: {total_loss}, Acc: {total_acc}")
    wandb.log({'epochs':  epoch + 1,
              'loss': total_loss,
              'acc': total_acc
                              })

Epoch: 1, Loss: 0.9431494474411011, Acc: 0.7179276315789473
Epoch: 2, Loss: 0.42344897985458374, Acc: 0.881578947368421
Epoch: 3, Loss: 0.313978374004364, Acc: 0.9152960526315789
Epoch: 4, Loss: 0.22633948922157288, Acc: 0.9391447368421052
Epoch: 5, Loss: 0.2635694146156311, Acc: 0.9226973684210525
Epoch: 6, Loss: 0.21754202246665955, Acc: 0.9383223684210525
Epoch: 7, Loss: 0.17074167728424072, Acc: 0.9613486842105262
Epoch: 8, Loss: 0.30612286925315857, Acc: 0.9473684210526315
Epoch: 9, Loss: 0.16001418232917786, Acc: 0.9720394736842105
Epoch: 10, Loss: 0.11227972060441971, Acc: 0.9671052631578947


In [14]:
# save
torch.save(model.state_dict(), "/content/drive/MyDrive/PersianMnistTL.pth")